In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import h5py

import matplotlib.pyplot as plt

In [2]:
ds_era5 = xr.load_dataset('data/perdigao_era5_2020.nc')
ds_era5['vel100'] = np.sqrt(ds_era5['u100'] ** 2 +  ds_era5['v100'] ** 2)
ds_era5['vel100'].attrs = {'long_name': '100 meter horizontal wind speed', 'units': 'm/s'}

In [3]:
ds_low_res_pre = xr.load_dataset('data/perdigao_low_res_1H_2020.nc')
ds_high_res_pre = xr.load_dataset('data/perdigao_high_res_1H_2020.nc')

In [4]:
#check if there is nan
for var_name in ds_era5.data_vars:
    print("ds_era5 ", var_name, np.isnan(ds_era5[var_name]).any().to_numpy())
print('\n')    

#check if there is nan
for var_name in ds_low_res_pre.data_vars:
    print("ds_low_res_pre ", var_name, np.isnan(ds_low_res_pre[var_name]).any().to_numpy())
print('\n')        

#check if there is nan
for var_name in ds_high_res_pre.data_vars:
    print("ds_high_res_pre ", var_name, np.isnan(ds_high_res_pre[var_name]).any().to_numpy())

ds_era5  u100 False
ds_era5  v100 False
ds_era5  t2m False
ds_era5  i10fg False
ds_era5  vel100 False


ds_low_res_pre  absolute_height False
ds_low_res_pre  std True
ds_low_res_pre  temp True
ds_low_res_pre  u True
ds_low_res_pre  v True
ds_low_res_pre  vel True


ds_high_res_pre  absolute_height True
ds_high_res_pre  std True
ds_high_res_pre  temp True
ds_high_res_pre  u True
ds_high_res_pre  v True
ds_high_res_pre  vel True


In [5]:
# Check the pattern where nan shows up
# The pattern should be that, for a given time, we see nan on all sites for all variables

low_res_nan_t_idx_list = []
for var_name in ds_low_res_pre.data_vars:
    if var_name == "absolute_height":
        continue
    print("Doing ", var_name, " for ds_low_res_pre")
    mask_low = np.isnan(ds_low_res_pre[var_name])
    nan_idx_tuple = np.where(mask_low == True)
    nan_t_idx = np.unique(nan_idx_tuple[0])
    print(nan_idx_tuple[0].shape, nan_t_idx.shape)
    low_res_nan_t_idx_list.append(nan_t_idx)
for i in range(len(low_res_nan_t_idx_list) -1):
    assert(np.array_equal(low_res_nan_t_idx_list[i], low_res_nan_t_idx_list[i+1]))
    
high_res_nan_t_idx_list = []
for var_name in ds_high_res_pre.data_vars:
    if var_name == "absolute_height":
        continue
    print("Doing ", var_name, " for ds_high_res_pre")
    mask_high = np.isnan(ds_high_res_pre[var_name])
    nan_idx_tuple = np.where(mask_high == True)
    nan_t_idx = np.unique(nan_idx_tuple[0])
    print(nan_idx_tuple[0].shape, nan_t_idx.shape)
    high_res_nan_t_idx_list.append(nan_t_idx)
for i in range(len(high_res_nan_t_idx_list) -1):
    assert(np.array_equal(high_res_nan_t_idx_list[i], high_res_nan_t_idx_list[i+1]))    

Doing  std  for ds_low_res_pre
(1446912,) (157,)
Doing  temp  for ds_low_res_pre
(1446912,) (157,)
Doing  u  for ds_low_res_pre
(1446912,) (157,)
Doing  v  for ds_low_res_pre
(1446912,) (157,)
Doing  vel  for ds_low_res_pre
(1446912,) (157,)
Doing  std  for ds_high_res_pre
(9363456,) (254,)
Doing  temp  for ds_high_res_pre
(9363456,) (254,)
Doing  u  for ds_high_res_pre
(9363456,) (254,)
Doing  v  for ds_high_res_pre
(9363456,) (254,)
Doing  vel  for ds_high_res_pre
(9363456,) (254,)


In [6]:
# Data preprocessing
# In this step we try to remove those timeslices where we only have nan. This is critical in DL

# First find the union of nan_t_idx of low/high res input ds
print("Low, ", low_res_nan_t_idx_list[0].shape, low_res_nan_t_idx_list[0])
print("High, ", high_res_nan_t_idx_list[0].shape, high_res_nan_t_idx_list[0])
nan_t_idx_union = np.union1d(low_res_nan_t_idx_list[0], high_res_nan_t_idx_list[0])
print("Union, ", nan_t_idx_union.shape, nan_t_idx_union)

Low,  (157,) [4165 4166 4167 4168 4169 4170 4171 4172 4173 4174 4175 4189 4190 4191
 4192 4193 4194 4195 4196 4197 4198 4199 4453 4454 4455 4456 4457 4458
 4459 4460 4461 4462 4463 4476 4477 4478 4479 4480 4481 4482 4483 4484
 4485 4486 4487 4500 4501 4502 4503 4504 4505 4506 4507 4508 4509 4510
 4511 4718 4719 4720 4721 4722 4723 4724 4725 4726 4727 4741 4742 4743
 4744 4745 4746 4747 4748 4749 4750 4751 4764 4765 4766 4767 4768 4769
 4770 4771 4772 4773 4774 4775 4788 4789 4790 4791 4792 4793 4794 4795
 4796 4797 4798 4799 4813 4814 4815 4816 4817 4818 4819 4820 4821 4822
 4823 4837 4838 4839 4840 4841 4842 4843 4844 4845 4846 4847 4981 4982
 4983 4984 4985 4986 4987 4988 4989 4990 4991 5053 5054 5055 5056 5057
 5058 5059 5060 5061 5062 5063 5221 5222 5223 5224 5225 5226 5227 5228
 5229 5230 5231]
High,  (254,) [1608 1609 1610 1611 1612 1613 1614 1615 1616 1617 1618 1619 1620 1621
 1622 1623 1624 1625 1626 1627 1628 1629 1630 1631 1944 1945 1946 1947
 1948 1949 1950 1951 1952 1953 19

In [7]:
# Then drop those data in both ds

print(ds_low_res_pre)
ds_low_res = ds_low_res_pre.drop_isel(time=nan_t_idx_union)
print(ds_low_res)
print('\n\n\n')
print(ds_high_res_pre)
ds_high_res = ds_high_res_pre.drop_isel(time=nan_t_idx_union)
print(ds_high_res)

<xarray.Dataset>
Dimensions:          (time: 8784, yf: 96, xf: 96)
Coordinates:
  * time             (time) datetime64[ns] 2020-01-01 ... 2020-12-31T23:00:00
    height           float32 100.0
  * xf               (xf) float64 7.76e+03 7.92e+03 ... 2.28e+04 2.296e+04
  * yf               (yf) float64 7.76e+03 7.92e+03 ... 2.28e+04 2.296e+04
Data variables:
    absolute_height  (time, yf, xf) float32 258.1 258.5 254.7 ... 324.4 310.8
    std              (time, yf, xf) float32 0.09803 0.1249 ... 1.039 0.9158
    temp             (time, yf, xf) float32 284.3 284.4 284.3 ... 278.9 278.7
    u                (time, yf, xf) float32 -2.41 -2.445 -2.446 ... 7.012 6.127
    v                (time, yf, xf) float32 0.3384 0.3794 ... -4.142 -3.613
    vel              (time, yf, xf) float32 2.436 2.476 2.477 ... 8.179 7.152
Attributes:
    site:         Perdigao, Portugal
    description:  160m x 160m x 40m LES simulation
    copyright:    GE Renewable Energy
<xarray.Dataset>
Dimensions:         

In [8]:
# Next make sure there is no nan anymore

for var_name in ds_low_res.data_vars:
    print("ds_low_res ", var_name, np.isnan(ds_low_res[var_name]).any().to_numpy())
for var_name in ds_high_res.data_vars:
    print("ds_high_res ", var_name, np.isnan(ds_high_res[var_name]).any().to_numpy())

ds_low_res  absolute_height False
ds_low_res  std False
ds_low_res  temp False
ds_low_res  u False
ds_low_res  v False
ds_low_res  vel False
ds_high_res  absolute_height False
ds_high_res  std False
ds_high_res  temp False
ds_high_res  u False
ds_high_res  v False
ds_high_res  vel False


In [9]:
# Finally save to disk

ds_low_res.to_netcdf('processed_data/perdigao_low_res_1H_2020.nc')
ds_high_res.to_netcdf('processed_data/perdigao_high_res_1H_2020.nc')

In [10]:
# Make sure we can open it again and it is the same

ds_low_res_reopen = xr.load_dataset('processed_data/perdigao_low_res_1H_2020.nc')
ds_high_res_reopen = xr.load_dataset('processed_data/perdigao_high_res_1H_2020.nc')
print(ds_low_res.equals(ds_low_res_reopen))
print(ds_high_res.equals(ds_high_res_reopen))

True
True


# Compute the statistics of low and high resolution data for GAN

In [11]:
mean_lr_u = ds_low_res["u"].mean().to_numpy()
mean_lr_v = ds_low_res["v"].mean().to_numpy()
mean_hr_u = ds_high_res["u"].mean().to_numpy()
mean_hr_v = ds_high_res["v"].mean().to_numpy()

In [12]:
print(mean_lr_u)
print(mean_lr_v)
print(mean_hr_u)
print(mean_hr_v)

0.7051484
-1.0147774
0.701198
-1.0068085


In [13]:
stddev_lr_u = ds_low_res["u"].std().to_numpy()
stddev_lr_v = ds_low_res["v"].std().to_numpy()
stddev_hr_u = ds_high_res["u"].std().to_numpy()
stddev_hr_v = ds_high_res["v"].std().to_numpy()

In [14]:
print(stddev_lr_u)
print(stddev_lr_v)
print(stddev_hr_u)
print(stddev_hr_v)

3.1869051
2.8827915
3.149407
2.8781955


# Standardrize the dataset for GAN

In [15]:
#First standardrize the hr dataset and test if the result is already standardrized

da_high_res_std_u = ( ds_high_res["u"] - mean_hr_u ) / stddev_hr_u
da_high_res_std_v = ( ds_high_res["v"] - mean_hr_v ) / stddev_hr_v
print(da_high_res_std_u.mean().to_numpy())
print(da_high_res_std_v.mean().to_numpy())
print(da_high_res_std_u.std().to_numpy())
print(da_high_res_std_v.std().to_numpy())

-1.8022865e-07
-9.072685e-07
0.9999977
1.0000002


In [16]:
#Next combine u and v component into a single array with two channels as the last dimension
#so weird... If I stack them with axis=-1, then result of std and mean will be incorrect!

np_hr_std_pre = np.stack((da_high_res_std_u.to_numpy(), da_high_res_std_v.to_numpy()), axis=0)
print(np_hr_std_pre.shape)

np_hr_std = np_hr_std_pre.transpose(1,2,3,0)
print(np_hr_std.shape)

print(np_hr_std.std(axis=(0,1,2)))
print(np_hr_std.mean(axis=(0,1,2)))

(2, 8520, 192, 192)
(8520, 192, 192, 2)
[0.9999977 1.0000002]
[-1.8022865e-07 -9.0726849e-07]


In [17]:
#Do things for lr dataset

da_low_res_std_u = ( ds_low_res["u"] - mean_lr_u ) / stddev_lr_u
da_low_res_std_v = ( ds_low_res["v"] - mean_lr_v ) / stddev_lr_v
print(da_low_res_std_u.mean().to_numpy())
print(da_low_res_std_v.mean().to_numpy())
print(da_low_res_std_u.std().to_numpy())
print(da_low_res_std_v.std().to_numpy())

#so weird... If I stack them with axis=-1, then result of std and mean will be incorrect!
np_lr_std_pre = np.stack((da_low_res_std_u.to_numpy(), da_low_res_std_v.to_numpy()), axis=0)
print(np_lr_std_pre.shape)

np_lr_std = np_lr_std_pre.transpose(1,2,3,0)
print(np_lr_std.shape)

print(np_lr_std.std(axis=(0,1,2)))
print(np_lr_std.mean(axis=(0,1,2)))

2.1677808e-08
-7.1720837e-07
1.0000008
1.0000011
(2, 8520, 96, 96)
(8520, 96, 96, 2)
[1.0000008 1.0000011]
[ 2.1677808e-08 -7.1720837e-07]


In [18]:
#Save mean/stddev of lr/hr data set as four 2-channel array

lr_mean = np.array([mean_lr_u, mean_lr_v])
hr_mean = np.array([mean_hr_u, mean_hr_v])
lr_stddev = np.array([stddev_lr_u, stddev_lr_v])
hr_stddev = np.array([stddev_hr_u, stddev_hr_v])

In [19]:
#Save everything into a hdf5 fileCopy1

with h5py.File('processed_data/np_gan_standard.h5', 'w') as hf:
    hf.create_dataset("np_lr",  data=np_lr_std)
    hf.create_dataset("np_hr",  data=np_hr_std)
    hf.create_dataset("np_lr_mean",  data=lr_mean)
    hf.create_dataset("np_hr_mean",  data=hr_mean)
    hf.create_dataset("np_lr_stddev",  data=lr_stddev)
    hf.create_dataset("np_hr_stddev",  data=hr_stddev)

In [20]:
#Test if we can reopen successfully and if the result is correct

with h5py.File('processed_data/np_gan_standard.h5', 'r') as hf:
    data_lr = hf['np_lr'][:]
    data_lr_mean = hf['np_lr_mean'][:]
    data_lr_stddev = hf['np_lr_stddev'][:]
    data_hr = hf['np_hr'][:]
    data_hr_mean = hf['np_hr_mean'][:]
    data_hr_stddev = hf['np_hr_stddev'][:]
    
print(np.array_equal(np_lr_std, data_lr))    
print(np.array_equal(np_hr_std, data_hr))

True
True


In [21]:
#Test if we can regenerate the old data. Use max to get rid of the floating point error

np_lr_reopen = data_lr * data_lr_stddev + data_lr_mean
print(np.max(np_lr_reopen[:,:,:,0] - ds_low_res["u"].to_numpy()))
print(np.max(np_lr_reopen[:,:,:,1] - ds_low_res["v"].to_numpy()))

np_hr_reopen = data_hr * data_hr_stddev + data_hr_mean
print(np.max(np_hr_reopen[:,:,:,0] - ds_high_res["u"].to_numpy()))
print(np.max(np_hr_reopen[:,:,:,1] - ds_high_res["v"].to_numpy()))

9.536743e-07
9.536743e-07
1.9073486e-06
9.536743e-07


In [22]:
print(data_lr.shape)
print(data_lr_mean)
print(data_lr_stddev)
print('\n\n')
print(data_hr.shape)
print(data_hr_mean)
print(data_hr_stddev)

(8520, 96, 96, 2)
[ 0.7051484 -1.0147774]
[3.1869051 2.8827915]



(8520, 192, 192, 2)
[ 0.701198  -1.0068085]
[3.149407  2.8781955]
